In [1]:
# Imports
import numpy as np
from numpy import random as ran
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import time
from PIL import Image

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [3]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from zipfile import ZipFile
with ZipFile('drive/MyDrive/Data.zip','r') as zipObj:
  zipObj.extractall('.')

with ZipFile('drive/MyDrive/tensorset_covnext_block5.zip','r') as zipObj:
  zipObj.extractall('.')

In [5]:
# dataset to load the feature tensors (storing them as numpy arrays takes much less memory)

class ImgDataset(Dataset):

    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        filename = 'tensorset/' + str(self.data[idx][0]) + '.npy'
        im1 = np.load(filename)
        filename = 'tensorset/' + str(self.data[idx][1]) + '.npy'
        im2 = np.load(filename)
        filename = 'tensorset/' + str(self.data[idx][2]) + '.npy'
        im3 = np.load(filename)

        im1 = torch.from_numpy(im1)
        im2 = torch.from_numpy(im2)
        im3 = torch.from_numpy(im3)

        return im1,im2,im3

In [6]:
# load trained model

model = torch.load("drive/MyDrive/covnext_epoch_5.pt")
for param in model.features.parameters():
    param.requires_grad = False

for param in model.avgpool.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False
model.to(device)
model.eval()

ConvNeXt(
  (features): Sequential(
    (0): Sequential(
      (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
          (1): Permute()
          (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=768, out_features=3072, bias=True)
          (4): GELU()
          (5): Linear(in_features=3072, out_features=768, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
          (1): Permute()
          (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_featu

In [7]:
# create test_dataset and dataloader

fname = 'Data/'
food = fname + 'food/'
test = np.loadtxt(fname + "test_triplets.txt", dtype=str)

test_dataset = ImgDataset(test)

testloader = DataLoader(test_dataset, batch_size=16,
                        shuffle=False, num_workers=0, pin_memory=True)

In [8]:
# CosineSimilarity is the criterion used to decide if the second or the third
# is more similar to the first image

cos = nn.CosineSimilarity(dim=1, eps=1e-6)

def compare(x,y,z):
    t = time.time()
    pred = np.zeros((x.shape[0],),dtype=int)
    dist1 = cos(x,y)
    dist2 = cos(x,z)
    res = torch.gt(dist1,dist2)
    return res


In [9]:
# compute the predictions

output = torch.empty((0,), dtype=torch.bool).to(device)

count = 0

for x_batch,y_batch,z_batch in testloader:
    with torch.no_grad():
      x_batch, y_batch, z_batch = x_batch.to(device), y_batch.to(device), z_batch.to(device)
      x = model(x_batch)
      y = model(y_batch)
      z = model(z_batch)
      res = compare(x,y,z)
      output = torch.cat((output,res))
      count += 1
      if count % 2000 == 0:
        print("Current step: ", count)

output = output.cpu().detach().numpy().astype(int)

Current step:  2000


In [10]:
np.savetxt('convnext_trip_5.txt',output,fmt='%i')

In [11]:
!cp "convnext_trip_5.txt" "drive/MyDrive/convnext_trip_5.txt"